# Query SRA metadata using Big Query

Using https://pandas-gbq.readthedocs.io/. 

Before you begin, you must create a Google Cloud Platform project. Use the BigQuery sandbox to try the service for free.

If you do not provide any credentials, this module attempts to load credentials from the environment. If no credentials are found, pandas-gbq prompts you to open a web browser, where you can grant it permissions to access your cloud resources. These credentials are only used locally.

In [7]:
import pandas_gbq
import pandas as pd

ModuleNotFoundError: No module named 'pandas_gbq'

In [2]:
# Construct query
# Here we are using the bioproject PRJNA523380 for the CCLE cell lines
# I only want to get the RNA-seq data 
query = """
SELECT * FROM `nih-sra-datastore.sra.metadata` 
WHERE bioproject = 'PRJNA523380' AND assay_type = 'RNA-Seq'
LIMIT 5000
"""

In [5]:
# Query the metadata table in BigQuery
# Might take some time
# Returns a dataframe
df = pd.read_gbq(query, dialect="standard")

ImportError: Missing optional dependency 'pandas-gbq'. pandas-gbq is required to load data from Google BigQuery. See the docs: https://pandas-gbq.readthedocs.io. Use pip or conda to install pandas-gbq.

# Goal here is to curate metadata for the results of the query
I specifically want a dataframe with the desired metadata as well as a dictionary with the same information.

Hoping to be able to use the dictionary to add the metadata to the objects in the the GCS bucket 

In [ ]:
#  TODO:: refactor these functions to be more generic
# Also improve efficiency by not iterating over the entire dataframe
# Maybe use a dictionary comprehension instead

def convert_array_to_dict(arr):
    result_dict = {}
    desired_keys = ['bases', 'bytes', 'run_file_create_date', 'disease_sam', 'disease_stage_sam_s_dpl172', 'tissue_sam']
    for item in arr:
        if isinstance(item, dict):
            key = item.get('k')
            value = item.get('v')
            if key in desired_keys:
                if key == 'bytes':
                    result_dict['size_in_bytes'] = int(value)
                    result_dict['size_in_GB'] = round(float(value) / 1000000000, 2)
                elif key == 'run_file_create_date':
                    result_dict[key] = str(value)
                else:
                    if key in result_dict:
                        if isinstance(result_dict[key], list):
                            result_dict[key].append(value)
                        else:
                            result_dict[key] = [result_dict[key], value]
                    else:
                        result_dict[key] = value
    return result_dict

def convert_row_to_dict(row):
    result_dict = {}
    for column in row.index:
        if column == 'attributes':
            result_dict.update(convert_array_to_dict(row[column]))
        elif column == 'releasedate':
            # convert type Timestamp to string
            result_dict[column] = str(row[column])
        elif column == 'run_file_create_date':
            result_dict[column] = str(row[column])
        else:
            result_dict[column] = row[column]
    return result_dict


def convert_dataframe_to_dict(df):
    result_dict = {}
    for i in range(len(df)):
        result_dict[df['acc'][i]] = convert_row_to_dict(df.iloc[i])
    return result_dict


In [ ]:
# choose columns
columns = ['acc', 'sample_name', 'sample_acc', 'experiment',  'library_name', 'sra_study', 'center_name', 
'platform', 'assay_type', 'librarysource', 'organism', 'releasedate']

# subset the dataframe to only include the columns we want
df_ = df[columns + ['attributes']].copy()

# # for each column in columns, print out the number of unique values, and then the first 5 unique values
for col in columns:
    # print(f"For {col}, there are: {df[col].nunique()} unique values. \nExamples: {df[col].unique()[0:5]}\n")
    print(f"{col} has {df[col].nunique()} unique values. \nExamples: {df[col].unique()[0:5]}\n")

In [ ]:
# Subset df to df_ using columns but also include the 'attributes' column
dict_metadata = convert_dataframe_to_dict(df_)
dict_metadata

In [ ]:
# convert dict to pandas dataframe
df_metadata = pd.DataFrame.from_dict(dict_metadata, orient='index')
# rename the 'acc' column to 'run_accession'
df_metadata.rename(columns={'acc': 'run_accession'}, inplace=True)
df_metadata

In [ ]:
# # save the dataframe to a csv file 
# df_metadata.to_csv('../../metadata/sra_metadata.csv', index=False)

# # save the dictionary to a json file
# import json
# with open('../../metadata/sra_metadata.json', 'w') as fp:
#     json.dump(dict_metadata, fp)